# **Preprocessing avant utilisation des modèles**

Ce notebook vise à explorer rapidement la base de données pour avoir une idée de se composition, la répartition et éventuels déséquilibres entre les classes, les caractéristiques des textes à analyser. Cela est éventuellement l'occasion de détecter des erreurs de textes (encodage erroné au moment d'enregistrer/lire les demandes)

In [1]:
# Pour faciliter la mise à jour des fonctions écrites dans func_custom sans avoir à redémarrer le kernel
%load_ext autoreload
%autoreload 2

In [71]:
# Packages classiques
import pandas as pd

# Custom package
import func_custom as fc

# NLP
import unidecode
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [3]:
df = pd.read_excel("data/data_train.xlsx",
                    usecols = ["label", "message"])

# 1. Analyse, justification et étapes du preprocessing

Si certaines étapes sont standards et ne nécessite pas d'explication (mettre en .lower() par exemple), regardons certains points plus particulièrement

## 1.1 Application de .lower() : non commenté

## 1.2 Encodage des accents

In [17]:
first_message = df["message"][0].split()
data_encodage = [first_message[i] for i in [21, 37, 43, 46]]
print(data_encodage)
keyboard_encodage = ["prélèvement", "époux", "à", "impôts"]
print(keyboard_encodage)

['prélèvement', 'époux', 'à', 'impôts']
['prélèvement', 'époux', 'à', 'impôts']


In [18]:
print([t.encode("unicode_escape") for t in data_encodage])
print([t.encode("unicode_escape") for t in keyboard_encodage])
print("")
print([unicodedata.normalize("NFKC", t).encode("unicode_escape") for t in data_encodage])
print([unicodedata.normalize("NFKC", t).encode("unicode_escape") for t in keyboard_encodage])

[b'pre\\u0301le\\u0300vement', b'e\\u0301poux', b'a\\u0300', b'impo\\u0302ts']
[b'pr\\xe9l\\xe8vement', b'\\xe9poux', b'\\xe0', b'imp\\xf4ts']

[b'pr\\xe9l\\xe8vement', b'\\xe9poux', b'\\xe0', b'imp\\xf4ts']
[b'pr\\xe9l\\xe8vement', b'\\xe9poux', b'\\xe0', b'imp\\xf4ts']


Il faudra une étape de normalisation avec `unicodedata.normalize("NFKC", text)`

## 1.3 Gestion de la ponctuation

Les signes de ponctuations sont tous remplacés par un _espace_ pour anticiper sur les failles du tokenizer de nltk. Typiquement en cas d'erreur de phrase, d'absence d'espace, surtout autour de mots inconnus le tokenizer rate complètement. Exemple avec cette phrase qui contient `0000.L'XXXXX́e` :

In [55]:
message_test = df[df["message"].str.contains("0000.L'XXXXX́e")]["message"].values[0]
print(message_test)

Bonjour, J'ai divorcé en 0000.J'ai commencé à verser une pension alimentaire à partir de septembre 0000.L'XXXXX́e dernière j'ai donc versé un XXXXX́e pleine.Par rapport au prélèvement à la source comment cela ce passe-t-il sachant que j'aurais plus à déduire pour l'XXXXX́e dernière que 0000?actuellement je suis prélever par rapport à 0000?y-aura t-il un remboursement en fin d'XXXXX́e?Peut-on modifier le prélèvement à la source en cours d'XXXXX́e? D'avance merci Cordialement XXXXX XXXXX


In [56]:
tokens = word_tokenize(message_test, language = "french")

On constate un problème avec le tokenizer qui persisterait si on supprimait simplement la ponctuation, je préfère donc la remplacer par un espace.

In [60]:
tokens[10:20]

['pension',
 'alimentaire',
 'à',
 'partir',
 'de',
 'septembre',
 "0000.L'XXXXX́e",
 'dernière',
 "j'ai",
 'donc']

In [63]:
tokens = word_tokenize(fc.replace_punctuation_with_space(message_test), language = "french")

In [65]:
tokens[15:25]

['partir',
 'de',
 'septembre',
 '0000',
 'L',
 'XXXXX́e',
 'dernière',
 'j',
 'ai',
 'donc']

## 1.4 Tokenizer

J'utilise ici le tokenizer par défaut de nltk en français

## 1.5 Gestion des stopwords

En combinant tous les éléments précédents dans la fonction `preprocess_stopwords` dans `func_custom.py` étudions désormais le traitement des stopwords. 
Commençons pas ne prendre en compte que ceux de base dans nltk :

In [69]:
stopwords_french = set(stopwords.words('french'))

In [104]:
text = ' '.join(df["message"].dropna())
tokens_clean = fc.preprocess_stopwords(text, stopwords_french)
pd.DataFrame(tokens_clean).value_counts().head(15)

xxxxx           1777
taux            1323
revenus          712
prélèvement      670
bonjour          624
source           526
a                522
cordialement     413
plus             366
merci            365
€                353
déclaration      300
comment          271
faire            260
bien             256
dtype: int64

On constate que la simple inclusion des stopwords par défaut de nltk ne suffira pas, des éléments spécifiques au jeu de données (`xxxxx` et `0000` issus de pseudonymisation, `bonjour` et `merci` et mots de politesse du fait qu'il s'agit de message écrits par des particuliers à la DGFIP). On peut alors compléter de manière _ad hoc_ cette liste en regardant à l'oeil nu ces éléments. Si l'exerice consiste cependant à distinguer les messages "polis" des autres alors cette liste de stopwords n'est absolument pas pertinente.

TF-IDF aurait dans une certaine mesure pu tenir compte de ces mots largement présent dans le corpus et peu informatif, autant traiter ce problème à la racine ce qui limitera la taille des données à traiter par la suite.

In [117]:
stopwords_adhoc = {"à", "xxxxx", "bonjour", "a", "cordialement", "merci", "xxxxx́e", "xxxxx́", "k€", "donc", "car", "cette", "cela",
                  "être", "si", "même", "faire", "avoir", "remercie", "madame", "monsieur"}
stopwords_complete = stopwords_french.union(stopwords_adhoc)

In [110]:
tokens_clean = fc.preprocess_stopwords(text, stopwords_complete)
pd.DataFrame(tokens_clean).value_counts().head(10)

taux           1323
revenus         712
prélèvement     670
source          526
plus            366
déclaration     300
comment         271
bien            256
situation       255
mois            249
dtype: int64

## 1.6 Suppression des caractères de taille 1

Pour prendre en compte des caractères spéciaux ou des lettres uniques suite au tokenizer, qui n'apporteront pas d'informations

## 1.7 (Non implémenté) prise en compte des fautes d'orthographe

## 1.8 Stemmer

# 2. Agrégation dans une fonction unique et application

In [118]:
df["message_clean"] = df["message"].apply(lambda x : fc.preprocess_text(x, stopwords_complete))

In [119]:
message_test = df.sample(1)
print(message_test["message"].values[0])
print(message_test["message_clean"].values[0])

Bonjour, J'ai une question sur le prélèvement à la source. Je suis pacsée depuis 0000 mais l'XXXXX́e dernière nous avons choisi de faire une déclaration séparée. Je viens de faire une simulation pour les revenus de 0000. Si nous déclarons ensemble, le taux du foyer serait de 0000.0000% (cf ma pièce jointe). Comment cela se traduit-il sur le prélèvement ? car il est indiqué que ce taux s 'applique pour le foyer. va-t-on nous prélever 0000.0000 % sur chaque salaire ou bien sur un des deux ? Je vous remercie pour votre réponse. Cordialement, XXXXX XXXXX
question prélèvement source pacsée depuis dernière choisi déclaration séparée viens simulation revenus déclarons ensemble taux foyer cf pièce jointe comment traduit prélèvement indiqué taux applique foyer va prélever chaque salaire bien deux réponse


In [121]:
df.to_pickle("data/data_clean.pkl")

In [122]:
df.to_csv("data/data_clean.csv", 
                sep = ";",
                index = False)